In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from scipy.stats import norm, skew # 수치해석 기능(확률분포) 제공
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,f1_score,roc_curve,roc_auc_score

train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/천체 유형 분류/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/천체 유형 분류/test.csv')
sample_submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/천체 유형 분류/sample_submission.csv')

train_id = train['id']
test_id = test['id']

# 필요없는 Id 컬럼, Drop하자.
#train.drop('id',axis=1,inplace=True)
#test.drop('id',axis=1,inplace=True)

# 'Id'를 제거한 DataFrame의 Shape 확인
print("The train data size after dropping ID feature is : {} ".format(train.shape))
print("The test data size after dropping Id feature is : {} ".format(test.shape))

unique_labels = train['type'].unique()
label_dict = {val: i for i, val in enumerate(unique_labels)}
i2lb = {v:k for k, v in label_dict.items()}

The train data size after dropping ID feature is : (199991, 23) 
The test data size after dropping Id feature is : (10009, 22) 


In [0]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [0]:
col_list = ['psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i','psfMag_z', 
            'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i','fiberMag_z', 
            'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i','petroMag_z',
            'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i','modelMag_z']

In [0]:
k=5
for idx,col in enumerate (col_list):
  for j in range(k):
    train['%s_%s*%s'%(col[:3],
                      col[-1],
                      col_list[idx+j][-1])] = train[col] * train[col_list[idx+j]]
    test['%s_%s*%s'%(col[:3],
                     col[-1],
                     col_list[idx+j][-1])] = test[col] * test[col_list[idx+j]]
  k-=1
  if k==0:
    k=5

In [0]:
test_id=test['id']
test = test.drop(columns=['fiberID', 'id'])

train_x = train.drop(columns=['id','type_num', 'fiberID','type'], axis=1)
train_y = train['type_num']
test_x = test

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_x, train_y, \
                                                  test_size=0.2, random_state=42, stratify = train_y)

In [32]:
#LGBM 1

from lightgbm import LGBMClassifier

lgbm_wrapper = LGBMClassifier(boosting_type='gbdt', num_leaves=800, max_depth=-1, learning_rate=0.01, min_data_in_leaf = 5,
                             n_estimators=10000, subsample_for_bin=250000, objective='multiclass', min_split_gain=0.34, reg_alpha = 0.01, reg_lambda = 0.01,
                             min_child_weight=0.005, min_child_samples=17,n_jobs=-1, random_state=123)

evals =[(X_train,y_train),(X_valid,y_valid)]
lgbm_wrapper.fit(X_train,y_train,early_stopping_rounds=20,
                 eval_metric='logloss',eval_set=evals,verbose=True)



[1]	training's multi_logloss: 2.2359	training's multi_logloss: 2.2359	valid_1's multi_logloss: 2.24628	valid_1's multi_logloss: 2.24628
Training until validation scores don't improve for 20 rounds.
[2]	training's multi_logloss: 2.18239	training's multi_logloss: 2.18239	valid_1's multi_logloss: 2.20093	valid_1's multi_logloss: 2.20093
[3]	training's multi_logloss: 2.13428	training's multi_logloss: 2.13428	valid_1's multi_logloss: 2.15995	valid_1's multi_logloss: 2.15995
[4]	training's multi_logloss: 2.09008	training's multi_logloss: 2.09008	valid_1's multi_logloss: 2.12212	valid_1's multi_logloss: 2.12212
[5]	training's multi_logloss: 2.04897	training's multi_logloss: 2.04897	valid_1's multi_logloss: 2.08663	valid_1's multi_logloss: 2.08663
[6]	training's multi_logloss: 2.01035	training's multi_logloss: 2.01035	valid_1's multi_logloss: 2.05323	valid_1's multi_logloss: 2.05323
[7]	training's multi_logloss: 1.97389	training's multi_logloss: 1.97389	valid_1's multi_logloss: 2.02169	valid_1

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.01, max_depth=-1,
               min_child_samples=17, min_child_weight=0.005, min_data_in_leaf=5,
               min_split_gain=0.34, n_estimators=10000, n_jobs=-1,
               num_leaves=800, objective='multiclass', random_state=123,
               reg_alpha=0.01, reg_lambda=0.01, silent=True, subsample=1.0,
               subsample_for_bin=250000, subsample_freq=0)